# Pose Error Project
### (Transformer-Encoder)

In [33]:
import logging

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Loading packages ...")
import os
import sys
import time
import pickle
import json

# 3rd party packages

#from tqdm import tqdm
# since we are using it in jupyter notebook
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# Project modules
from options import Options
from running import setup, pipeline_factory, validate, check_progress, NEG_METRICS
from utils import utils
from datasets.data import data_factory, Normalizer
from datasets.datasplit import split_dataset
from models.ts_transformer import model_factory
from models.loss import get_loss_module
from optimizers import get_optimizer

import parser

import time

2023-05-23 11:26:29,503 | INFO : Loading packages ...


# Setup Configurations

d_model is dimension of representations used as input to the multi-head attention.
which is the same as the output.

d_model is the same size as the embedding size.

dim_feedforward is the dimension of the Fully-Connected layer

In [48]:
# setting 3: Test_only
text = "--output_dir ../experiments --comment 'poseErrorPred_test' \
        --name poseErrorPred_test  --records_file Regression_records.xls \
        --data_dir ../data/SenseTimeV4_Selective/ --data_class pose \
        --load_model ../experiments/poseErrorPred_fromScratch_Regression_Selective_3_64_2023-05-23_10-03-58_E2H/checkpoints/model_best.pth \
        --test_only testset --test_ratio 0.2\
        --num_layers 3  --num_heads 8 --d_model 64 --dim_feedforward 256 \
        --batch_size 1 --gpu 1 --task regression"

In [49]:
# Process the setting string
# Generate the config variable
input_text = text.split()
args = Options().parse(input_text)
config = setup(args)

2023-05-23 11:31:55,700 | INFO : Stored configuration file in '../experiments/poseErrorPred_test_2023-05-23_11-31-55_CGF'


In [50]:
config

{'config_filepath': None,
 'output_dir': '../experiments/poseErrorPred_test_2023-05-23_11-31-55_CGF',
 'data_dir': '../data/SenseTimeV4_Selective/',
 'load_model': '../experiments/poseErrorPred_fromScratch_Regression_Selective_3_64_2023-05-23_10-03-58_E2H/checkpoints/model_best.pth',
 'resume': False,
 'change_output': False,
 'save_all': False,
 'experiment_name': 'poseErrorPred_test',
 'comment': "'poseErrorPred_test'",
 'no_timestamp': False,
 'records_file': 'Regression_records.xls',
 'console': False,
 'print_interval': 1,
 'gpu': '1',
 'n_proc': -1,
 'num_workers': 0,
 'seed': None,
 'limit_size': None,
 'test_only': 'testset',
 'data_class': 'pose',
 'labels': None,
 'test_from': None,
 'test_ratio': 0.2,
 'val_ratio': 0.2,
 'pattern': None,
 'val_pattern': None,
 'test_pattern': None,
 'normalization': 'standardization',
 'norm_from': None,
 'subsample_factor': None,
 'task': 'regression',
 'masking_ratio': 0.15,
 'mean_mask_length': 3,
 'mask_mode': 'separate',
 'mask_distribu

# Setup Logger

In [51]:
total_epoch_time = 0
total_eval_time = 0

total_start_time = time.time()

In [52]:
# Add file logging besides stdout
file_handler = logging.FileHandler(os.path.join(config['output_dir'], 'output.log'))
logger.addHandler(file_handler)

logger.info('Running:\n{}\n'.format(' '.join(sys.argv)))  # command used to run

2023-05-23 11:31:56,498 | INFO : Running:
/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/tianyi/.local/share/jupyter/runtime/kernel-01e6773c-4932-4fb4-b11f-6320601b3559.json



# Setup Training Device

In [53]:
if config['seed'] is not None:
    torch.manual_seed(config['seed'])

device = torch.device('cuda' if (torch.cuda.is_available() and config['gpu'] != '-1') else 'cpu')
logger.info("Using device: {}".format(device))
if device == 'cuda':
    logger.info("Device index: {}".format(torch.cuda.current_device()))

2023-05-23 11:31:56,866 | INFO : Using device: cuda


# Prepare Data

In [54]:
 # Build data
logger.info("Loading and preprocessing data ...")
data_class = data_factory[config['data_class']]
my_data = data_class(config['data_dir'], 
                     pattern=config['pattern'], 
                     n_proc=config['n_proc'], 
                     limit_size=config['limit_size'], 
                     config=config)
feat_dim = my_data.feature_df.shape[1]  # dimensionality of data features
if config['task'] == 'classification':
    validation_method = 'StratifiedShuffleSplit'
    labels = my_data.labels_df.values.flatten()
else:
    validation_method = 'ShuffleSplit'
    labels = None
    
# Modify for the pose error pred
validation_method = 'PoseErrorTimeSplit'

2023-05-23 11:31:57,283 | INFO : Loading and preprocessing data ...
2023-05-23 11:31:57,285 | INFO : Loading 69 datasets files using 32 parallel processes ...


# Split dataset

In [55]:
# Split dataset
test_data = my_data
test_indices = None  # will be converted to empty list in `split_dataset`, if also test_set_ratio == 0
val_data = my_data
val_indices = []
if config['test_pattern']:  # used if test data come from different files / file patterns
    test_data = data_class(config['data_dir'], pattern=config['test_pattern'], n_proc=-1, config=config)
    test_indices = test_data.all_IDs
if config['test_from']:  # load test IDs directly from file, if available, otherwise use `test_set_ratio`. Can work together with `test_pattern`
    test_indices = list(set([line.rstrip() for line in open(config['test_from']).readlines()]))
    try:
        test_indices = [int(ind) for ind in test_indices]  # integer indices
    except ValueError:
        pass  # in case indices are non-integers
    logger.info("Loaded {} test IDs from file: '{}'".format(len(test_indices), config['test_from']))
if config['val_pattern']:  # used if val data come from different files / file patterns
    val_data = data_class(config['data_dir'], pattern=config['val_pattern'], n_proc=-1, config=config)
    val_indices = val_data.all_IDs

In [56]:
# Note: currently a validation set must exist, either with `val_pattern` or `val_ratio`
# Using a `val_pattern` means that `val_ratio` == 0 and `test_ratio` == 0
if config['val_ratio'] > 0:
    train_indices, val_indices, test_indices = split_dataset(data_indices=my_data.all_IDs,
                                                             validation_method=validation_method,
                                                             n_splits=1,
                                                             validation_ratio=config['val_ratio'],
                                                             test_set_ratio=config['test_ratio'],  # used only if test_indices not explicitly specified
                                                             test_indices=test_indices,
                                                             random_seed=1337,
                                                             labels=labels)
    train_indices = train_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
    val_indices = val_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
else:
    train_indices = my_data.all_IDs
    if test_indices is None:
        test_indices = []

In [57]:
logger.info("{} \t samples may be used for training".format(len(train_indices)))
logger.info("{} \t samples will be used for validation".format(len(val_indices)))
logger.info("{} \t samples will be used for testing".format(len(test_indices)))

2023-05-23 11:32:18,904 | INFO : 70675 	 samples may be used for training
2023-05-23 11:32:18,905 | INFO : 17709 	 samples will be used for validation
2023-05-23 11:32:18,905 | INFO : 22096 	 samples will be used for testing


In [58]:
with open(os.path.join(config['output_dir'], 'data_indices.json'), 'w') as f:
    try:
        json.dump({'train_indices': list(map(int, train_indices)),
                   'val_indices': list(map(int, val_indices)),
                   'test_indices': list(map(int, test_indices))}, f, indent=4)
    except ValueError:  # in case indices are non-integers
        json.dump({'train_indices': list(train_indices),
                   'val_indices': list(val_indices),
                   'test_indices': list(test_indices)}, f, indent=4)


In [59]:
# Pre-process features
normalizer = None
if config['norm_from']:
    with open(config['norm_from'], 'rb') as f:
        norm_dict = pickle.load(f)
    normalizer = Normalizer(**norm_dict)
elif config['normalization'] is not None:
    normalizer = Normalizer(config['normalization'])
    my_data.feature_df.loc[train_indices] = normalizer.normalize(my_data.feature_df.loc[train_indices])
    if not config['normalization'].startswith('per_sample'):
        # get normalizing values from training set and store for future use
        norm_dict = normalizer.__dict__
        with open(os.path.join(config['output_dir'], 'normalization.pickle'), 'wb') as f:
            pickle.dump(norm_dict, f, pickle.HIGHEST_PROTOCOL)
if normalizer is not None:
    if len(val_indices):
        val_data.feature_df.loc[val_indices] = normalizer.normalize(val_data.feature_df.loc[val_indices])
    if len(test_indices):
        test_data.feature_df.loc[test_indices] = normalizer.normalize(test_data.feature_df.loc[test_indices])


# Create model

In [60]:
# Create model
logger.info("Creating model ...")
model = model_factory(config, my_data)

if config['freeze']:
    for name, param in model.named_parameters():
        if name.startswith('output_layer'):
            param.requires_grad = True
        else:
            param.requires_grad = False

logger.info("Model:\n{}".format(model))
logger.info("Total number of parameters: {}".format(utils.count_parameters(model)))
logger.info("Trainable parameters: {}".format(utils.count_parameters(model, trainable=True)))


2023-05-23 11:32:25,070 | INFO : Creating model ...
2023-05-23 11:32:25,076 | INFO : Model:
TSTransformerEncoderClassiregressor(
  (project_inp): Linear(in_features=12, out_features=64, bias=True)
  (pos_enc): FixedPositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerBatchNormEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout

# Initialize optimizer

In [61]:
# Initialize optimizer
if config['global_reg']:
    weight_decay = config['l2_reg']
    output_reg = None
else:
    weight_decay = 0
    output_reg = config['l2_reg']

optim_class = get_optimizer(config['optimizer'])
optimizer = optim_class(model.parameters(), lr=config['lr'], weight_decay=weight_decay)

start_epoch = 0
lr_step = 0  # current step index of `lr_step`
lr = config['lr']  # current learning step
# Load model and optimizer state
if args.load_model:
    model, optimizer, start_epoch = utils.load_model(model, config['load_model'], optimizer, config['resume'],
                                                     config['change_output'],
                                                     config['lr'],
                                                     config['lr_step'],
                                                     config['lr_factor'])
model.to(device)

loss_module = get_loss_module(config)

Loaded model from ../experiments/poseErrorPred_fromScratch_Regression_Selective_3_64_2023-05-23_10-03-58_E2H/checkpoints/model_best.pth. Epoch: 10


In [62]:

if config['test_only'] == 'testset':  # Only evaluate and skip training
    dataset_class, collate_fn, runner_class = pipeline_factory(config)
    test_dataset = dataset_class(test_data, test_indices)

    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=config['batch_size'],
                             shuffle=False,
                             num_workers=config['num_workers'],
                             pin_memory=True,
                             collate_fn=lambda x: collate_fn(x, max_len=model.max_len))
    start_compute_time = time.time()
    test_evaluator = runner_class(model, test_loader, device, loss_module,
                                        print_interval=config['print_interval'], console=config['console'])
    aggr_metrics_test, per_batch_test = test_evaluator.evaluate(keep_all=True)
    end_compute_time = time.time()
    elapsed_compute_time = end_compute_time - start_compute_time
    print("Elapsed Time: {}s, Single Inferece Time {}s".format(elapsed_compute_time, elapsed_compute_time/len(test_loader)))
    
    print_str = 'Test Summary: '
    for k, v in aggr_metrics_test.items():
        print_str += '{}: {} | '.format(k, v)
    logger.info(print_str)
    #return

Evaluating   0.0% | batch:         0 of     22096	|	loss: 0.0884673
Evaluating   0.0% | batch:         1 of     22096	|	loss: 0.580951
Evaluating   0.0% | batch:         2 of     22096	|	loss: 0.203517
Evaluating   0.0% | batch:         3 of     22096	|	loss: 0.106365
Evaluating   0.0% | batch:         4 of     22096	|	loss: 1.13933
Evaluating   0.0% | batch:         5 of     22096	|	loss: 0.0133893
Evaluating   0.0% | batch:         6 of     22096	|	loss: 0.158502
Evaluating   0.0% | batch:         7 of     22096	|	loss: 0.791462
Evaluating   0.0% | batch:         8 of     22096	|	loss: 0.654067
Evaluating   0.0% | batch:         9 of     22096	|	loss: 0.46824
Evaluating   0.0% | batch:        10 of     22096	|	loss: 0.0804556
Evaluating   0.0% | batch:        11 of     22096	|	loss: 0.45611
Evaluating   0.1% | batch:        12 of     22096	|	loss: 0.303618
Evaluating   0.1% | batch:        13 of     22096	|	loss: 0.115589
Evaluating   0.1% | batch:        14 of     22096	|	loss: 0.32

Evaluating   0.8% | batch:       186 of     22096	|	loss: 0.750836
Evaluating   0.8% | batch:       187 of     22096	|	loss: 0.944893
Evaluating   0.9% | batch:       188 of     22096	|	loss: 0.873409
Evaluating   0.9% | batch:       189 of     22096	|	loss: 0.740929
Evaluating   0.9% | batch:       190 of     22096	|	loss: 0.103224
Evaluating   0.9% | batch:       191 of     22096	|	loss: 0.674958
Evaluating   0.9% | batch:       192 of     22096	|	loss: 0.919778
Evaluating   0.9% | batch:       193 of     22096	|	loss: 0.0390426
Evaluating   0.9% | batch:       194 of     22096	|	loss: 0.123448
Evaluating   0.9% | batch:       195 of     22096	|	loss: 0.237175
Evaluating   0.9% | batch:       196 of     22096	|	loss: 0.0155507
Evaluating   0.9% | batch:       197 of     22096	|	loss: 0.371874
Evaluating   0.9% | batch:       198 of     22096	|	loss: 0.0744432
Evaluating   0.9% | batch:       199 of     22096	|	loss: 2.17017
Evaluating   0.9% | batch:       200 of     22096	|	loss: 0.

Evaluating   1.4% | batch:       312 of     22096	|	loss: 0.0178002
Evaluating   1.4% | batch:       313 of     22096	|	loss: 0.242603
Evaluating   1.4% | batch:       314 of     22096	|	loss: 0.283022
Evaluating   1.4% | batch:       315 of     22096	|	loss: 0.384829
Evaluating   1.4% | batch:       316 of     22096	|	loss: 0.36509
Evaluating   1.4% | batch:       317 of     22096	|	loss: 0.390907
Evaluating   1.4% | batch:       318 of     22096	|	loss: 0.552607
Evaluating   1.4% | batch:       319 of     22096	|	loss: 0.422237
Evaluating   1.4% | batch:       320 of     22096	|	loss: 0.536808
Evaluating   1.5% | batch:       321 of     22096	|	loss: 0.448895
Evaluating   1.5% | batch:       322 of     22096	|	loss: 0.499676
Evaluating   1.5% | batch:       323 of     22096	|	loss: 0.389209
Evaluating   1.5% | batch:       324 of     22096	|	loss: 0.216863
Evaluating   1.5% | batch:       325 of     22096	|	loss: 0.108895
Evaluating   1.5% | batch:       326 of     22096	|	loss: 1.47

Evaluating   2.0% | batch:       440 of     22096	|	loss: 0.354095
Evaluating   2.0% | batch:       441 of     22096	|	loss: 0.25579
Evaluating   2.0% | batch:       442 of     22096	|	loss: 2.30895
Evaluating   2.0% | batch:       443 of     22096	|	loss: 1.05267
Evaluating   2.0% | batch:       444 of     22096	|	loss: 0.550098
Evaluating   2.0% | batch:       445 of     22096	|	loss: 0.185869
Evaluating   2.0% | batch:       446 of     22096	|	loss: 0.00177346
Evaluating   2.0% | batch:       447 of     22096	|	loss: 0.627619
Evaluating   2.0% | batch:       448 of     22096	|	loss: 0.1806
Evaluating   2.0% | batch:       449 of     22096	|	loss: 0.000452764
Evaluating   2.0% | batch:       450 of     22096	|	loss: 0.13442
Evaluating   2.0% | batch:       451 of     22096	|	loss: 0.380934
Evaluating   2.0% | batch:       452 of     22096	|	loss: 0.13435
Evaluating   2.1% | batch:       453 of     22096	|	loss: 0.415727
Evaluating   2.1% | batch:       454 of     22096	|	loss: 0.3159

Evaluating   2.6% | batch:       564 of     22096	|	loss: 0.469277
Evaluating   2.6% | batch:       565 of     22096	|	loss: 0.134498
Evaluating   2.6% | batch:       566 of     22096	|	loss: 0.625903
Evaluating   2.6% | batch:       567 of     22096	|	loss: 0.327957
Evaluating   2.6% | batch:       568 of     22096	|	loss: 0.505233
Evaluating   2.6% | batch:       569 of     22096	|	loss: 0.302476
Evaluating   2.6% | batch:       570 of     22096	|	loss: 0.923495
Evaluating   2.6% | batch:       571 of     22096	|	loss: 0.128351
Evaluating   2.6% | batch:       572 of     22096	|	loss: 0.413856
Evaluating   2.6% | batch:       573 of     22096	|	loss: 1.4175
Evaluating   2.6% | batch:       574 of     22096	|	loss: 0.0590776
Evaluating   2.6% | batch:       575 of     22096	|	loss: 0.338165
Evaluating   2.6% | batch:       576 of     22096	|	loss: 1.34835
Evaluating   2.6% | batch:       577 of     22096	|	loss: 0.711901
Evaluating   2.6% | batch:       578 of     22096	|	loss: 0.6739

...

Evaluating  98.5% | batch:     21755 of     22096	|	loss: 0.106838
Evaluating  98.5% | batch:     21756 of     22096	|	loss: 0.330919
Evaluating  98.5% | batch:     21757 of     22096	|	loss: 0.126991
Evaluating  98.5% | batch:     21758 of     22096	|	loss: 0.494774
Evaluating  98.5% | batch:     21759 of     22096	|	loss: 0.171016
Evaluating  98.5% | batch:     21760 of     22096	|	loss: 0.874416
Evaluating  98.5% | batch:     21761 of     22096	|	loss: 0.179271
Evaluating  98.5% | batch:     21762 of     22096	|	loss: 0.00188329
Evaluating  98.5% | batch:     21763 of     22096	|	loss: 1.17067
Evaluating  98.5% | batch:     21764 of     22096	|	loss: 1.25134
Evaluating  98.5% | batch:     21765 of     22096	|	loss: 0.0875354
Evaluating  98.5% | batch:     21766 of     22096	|	loss: 0.293977
Evaluating  98.5% | batch:     21767 of     22096	|	loss: 0.356813
Evaluating  98.5% | batch:     21768 of     22096	|	loss: 0.380396
Evaluating  98.5% | batch:     21769 of     22096	|	loss: 1.1

Evaluating  99.0% | batch:     21883 of     22096	|	loss: 0.229963
Evaluating  99.0% | batch:     21884 of     22096	|	loss: 0.686051
Evaluating  99.0% | batch:     21885 of     22096	|	loss: 0.13067
Evaluating  99.0% | batch:     21886 of     22096	|	loss: 1.07024
Evaluating  99.1% | batch:     21887 of     22096	|	loss: 1.32667
Evaluating  99.1% | batch:     21888 of     22096	|	loss: 0.0217493
Evaluating  99.1% | batch:     21889 of     22096	|	loss: 3.22158
Evaluating  99.1% | batch:     21890 of     22096	|	loss: 0.0273818
Evaluating  99.1% | batch:     21891 of     22096	|	loss: 0.0773923
Evaluating  99.1% | batch:     21892 of     22096	|	loss: 0.149509
Evaluating  99.1% | batch:     21893 of     22096	|	loss: 0.230781
Evaluating  99.1% | batch:     21894 of     22096	|	loss: 0.817236
Evaluating  99.1% | batch:     21895 of     22096	|	loss: 0.284275
Evaluating  99.1% | batch:     21896 of     22096	|	loss: 0.544778
Evaluating  99.1% | batch:     21897 of     22096	|	loss: 0.299

Evaluating  99.6% | batch:     22013 of     22096	|	loss: 0.489052
Evaluating  99.6% | batch:     22014 of     22096	|	loss: 0.798415
Evaluating  99.6% | batch:     22015 of     22096	|	loss: 0.157863
Evaluating  99.6% | batch:     22016 of     22096	|	loss: 1.34836
Evaluating  99.6% | batch:     22017 of     22096	|	loss: 0.115533
Evaluating  99.6% | batch:     22018 of     22096	|	loss: 0.23031
Evaluating  99.7% | batch:     22019 of     22096	|	loss: 0.117669
Evaluating  99.7% | batch:     22020 of     22096	|	loss: 0.51664
Evaluating  99.7% | batch:     22021 of     22096	|	loss: 0.94741
Evaluating  99.7% | batch:     22022 of     22096	|	loss: 0.231959
Evaluating  99.7% | batch:     22023 of     22096	|	loss: 0.0626323
Evaluating  99.7% | batch:     22024 of     22096	|	loss: 0.559265
Evaluating  99.7% | batch:     22025 of     22096	|	loss: 0.589274
Evaluating  99.7% | batch:     22026 of     22096	|	loss: 0.241915
Evaluating  99.7% | batch:     22027 of     22096	|	loss: 2.64063

2023-05-23 11:33:35,839 | INFO : Test Summary: epoch: None | loss: 0.6835912172003588 | 


Evaluating  99.9% | batch:     22078 of     22096	|	loss: 0.0159437
Evaluating  99.9% | batch:     22079 of     22096	|	loss: 0.120703
Evaluating  99.9% | batch:     22080 of     22096	|	loss: 0.723237
Evaluating  99.9% | batch:     22081 of     22096	|	loss: 0.2643
Evaluating  99.9% | batch:     22082 of     22096	|	loss: 0.26703
Evaluating  99.9% | batch:     22083 of     22096	|	loss: 0.0287896
Evaluating  99.9% | batch:     22084 of     22096	|	loss: 0.0711031
Evaluating 100.0% | batch:     22085 of     22096	|	loss: 0.00784863
Evaluating 100.0% | batch:     22086 of     22096	|	loss: 0.123777
Evaluating 100.0% | batch:     22087 of     22096	|	loss: 0.580109
Evaluating 100.0% | batch:     22088 of     22096	|	loss: 0.569431
Evaluating 100.0% | batch:     22089 of     22096	|	loss: 0.0709326
Evaluating 100.0% | batch:     22090 of     22096	|	loss: 0.511376
Evaluating 100.0% | batch:     22091 of     22096	|	loss: 0.285698
Evaluating 100.0% | batch:     22092 of     22096	|	loss: 0